In [61]:
import netsquid as ns

# simple protocl that sleeps for 100 nanoseconds:

In [62]:
from netsquid.protocols import Protocol

class WaitProtocol(Protocol):
    def run(self):
        print(f"Starting protcol at {ns.sim_time()}")
        yield self.await_timer(100)
        print(f"Ending protocl at {ns.sim_time()}")
        
ns.sim_reset()
protocol = WaitProtocol()
protocol.start()

Starting protcol at 0.0


WaitProtocol('WaitProtocol')

In [63]:
stats = ns.sim_run()

Ending protocl at 100.0


when protocol finsihes, sends FINISHED signal (Signals) to inform any lsitienting entiteisi

# ping pong example using protocols

In [64]:
from netsquid.protocols import NodeProtocol
from netsquid.components import QuantumChannel
from netsquid.nodes import Node, DirectConnection
from netsquid.qubits import qubitapi as qapi

class PingProtocol(NodeProtocol):
    def run(self):
        print(f"Starting ping at t={ns.sim_time()}")
        port = self.node.ports["port_to_channel"]
        qubit, = qapi.create_qubits(1)
        port.tx_output(qubit) # send qubit to POng
        while True:
            # wiat for qubit to be received back
            yield self.await_port_input(port)
            qubit = port.rx_input().items[0]
            m, prob = qapi.measure(qubit, ns.Z)
            labels_z = ("|0>", "|1>")
            print(f"{ns.sim_time()}: Pong event! {self.node.name} measured"
                  f"{labels_z[m]} with probability {prob:.2f}")
            port.tx_output(qubit) #send qubit to Pong
            
class PongProtocol(NodeProtocol):
    def run(self):
        print("Starting pong at t={}".format(ns.sim_time()))
        port = self.node.ports["port_to_channel"]
        while True:
            yield self.await_port_input(port)
            qubit = port.rx_input().items[0]
            m, prob = qapi.measure(qubit,ns.X)
            labels_x = ("|+>", "|->")
            print(f"{ns.sim_time()}: Ping event! {self.node.name} measured "
                  f"{labels_x[m]} with probability {prob:.2f}")
            port.tx_output(qubit) #send quibt to Ping

In [65]:
# running two porotocol on nodes of Ping and Pong entityt
# connect thru direct connection 
# assign them their protocol
ns.sim_reset()
ns.set_random_state(seed=42)

node_ping = Node("Ping", port_names=["port_to_channel"])
node_pong = Node("Pong", port_names=["port_to_channel"])

connection = DirectConnection("Connection",
                             QuantumChannel("Channel_LR", delay=10), 
                             QuantumChannel("Channel_RL", delay=10))

node_ping.ports["port_to_channel"].connect(connection.ports['A'])
node_pong.ports["port_to_channel"].connect(connection.ports["B"])
ping_protocol = PingProtocol(node_ping)
pong_protocol = PongProtocol(node_pong)

In [66]:
# runing simulation
ping_protocol.start()
pong_protocol.start()
stats = ns.sim_run(91)

Starting ping at t=0.0
Starting pong at t=0.0
10.0: Ping event! Pong measured |+> with probability 0.50
20.0: Pong event! Ping measured|1> with probability 0.50
30.0: Ping event! Pong measured |-> with probability 0.50
40.0: Pong event! Ping measured|1> with probability 0.50
50.0: Ping event! Pong measured |+> with probability 0.50
60.0: Pong event! Ping measured|0> with probability 0.50
70.0: Ping event! Pong measured |+> with probability 0.50
80.0: Pong event! Ping measured|1> with probability 0.50
90.0: Ping event! Pong measured |-> with probability 0.50


In [67]:
# what happesn if protocols are stopped or reset?
pong_protocol.stop()
stats = ns.sim_run()

100.0: Pong event! Ping measured|1> with probability 0.50


Now that the pong protocol is stopped the simulation can be run until there are no more events. In the previous run, pong sent a qubit back to ping, but it was not yet processed. In this run, the lingering qubit arrived at the ping protocol. The qubit is pinged back, but pong is stopped so does not process it. There was no one listening to the port, so now the qubit is lost. When pong is started again, nothing will happen, since there is no qubit to pass back and forth.

In [68]:
pong_protocol.start()
stats = ns.sim_run()

Starting pong at t=110.0


In [69]:
ping_protocol.reset()
stats = ns.sim_run(duration = 51)

Starting ping at t=110.0
120.0: Ping event! Pong measured |+> with probability 0.50
130.0: Pong event! Ping measured|1> with probability 0.50
140.0: Ping event! Pong measured |-> with probability 0.50
150.0: Pong event! Ping measured|0> with probability 0.50
160.0: Ping event! Pong measured |+> with probability 0.50


# teleporation exmaple using protocols

You may think that Alice could simply send a signal with the measurement results, to which Bob could listen to. However that would break locality; the message would have been sent faster than light to Bob! To avoid this possibility a node protocol (NodeProtocol) can be used. Such a protocol has access to only one node, and also ensures protocols can only signal to other protocols on that node i.e. at the same location. If a protocol should have access to a limited set of nodes a local protocol (LocalProtocol) can be used, or a regular protocol (LocalProtocol) if locality is not of concern.

In [70]:
from netsquid.protocols import NodeProtocol, Signals

class InitStateProtocol(NodeProtocol):
    def run(self):
        qubit, = qapi.create_qubits(1)
        mem_pos = self.node.qmemory.unused_positions[0]
        self.node.qmemory.put(qubit, mem_pos)
        self.node.qmemory.operate(ns.H, mem_pos)
        self.node.qmemory.operate(ns.S, mem_pos)
        self.send_signal(signal_label=Signals.SUCCESS, result=mem_pos)

Next Alice needs a protocol to perform her Bell measurement. Before she can do the measurement both the qubit she wants to teleport and the qubit that is entangled with Bob needs to be in her memory, so Alice needs to wait for both of them. In principle they can arrive in any order, so Alice will need to wait for both of them simultaneously. This can be done by combining the event expressions with the & (AND) operator, which waits until both expressions are triggered:

expression_and = yield expression1 & expression2
expression_or = yield expression3 | expression4

#The event expression that is returned is a copy of the event expression 
#that was yielded on. This new copied event expression has information on 
#which expression was triggered.

expression_or.first_term.value  # Is true if expression3 was triggered  
expression_or.second_term.value  # Is true if expression4 was triggered  
#list of all events that caused the expression to trigger:
expression_or.triggered_events    

Alice waits for a signal of the InitStateProtocol (i.e. her qubit being prepared), and until the entangled qubit gets placed on the memory. Once both have occurred she can do the measurement and send the measurement results to Bob.

In [71]:
from pydynaa import EventExpression

class BellMeasurementProtocol(NodeProtocol):
    def __init__(self, node, qubit_protocol):
        super().__init__(node)
        self.add_subprotocol(qubit_protocol, 'qprotocol')
        
    def run(self):
        qubit_initialized = False
        entanglement_ready = False
        while True:
            evexpr_signal = self.await_signal(
                sender= self.subprotocols['qprotocol'],
                signal_label=Signals.SUCCESS)
            evexpr_port = self.await_port_input(self.node.ports["qin_charlie"])
            expression = yield evexpr_signal | evexpr_port
            if expression.first_term.value:
                # first expression was triggered
                qubit_initialized = True
            else: 
                # second expression was triggerted
                entanglement_ready = True
            if qubit_initialized and entanglement_ready:
                # perform Bell measuremetn:
                self.node.qmemory.operate(ns.CNOT, [0,1])
                self.node.qmemory.operate(ns.H, 0)
                m, _ = self.node.qmemory.measure([0,1])
                # send measuremetn results to Bob:
                self.node.ports["cout_bob"].tx_output(m)
                self.send_signal(Signals.SUCCESS)
                print(f"{ns.sim_time():.1f} Alice received the entangled qubit, "
                      f"measured qubits & sending corrections")
                break

    def start(self):
        super().start()
        self.start_subprotocols()

Bob needs to perofmr coreections on qubit it receives from the soruce, so Bob also needs to wait for 2 things to occur:
- classical data he receives from Alice
- the entangled qubit that gets placed in Bob's memeory

In [74]:
class CorrectionProtocol(NodeProtocol):

    def __init__(self, node):
        super().__init__(node)

    def run(self):
        port_alice = self.node.ports["cin_alice"]
        port_charlie = self.node.ports["qin_charlie"]
        entanglement_ready = False
        meas_results = None
        while True:
            evexpr_port_a = self.await_port_input(port_alice)
            evexpr_port_c = self.await_port_input(port_charlie)
            expression = yield evexpr_port_a | evexpr_port_c
            if expression.first_term.value:
                meas_results = port_alice.rx_input().items
            else:
                entanglement_ready = True
            if meas_results is not None and entanglement_ready:
                if meas_results[0]:
                    self.node.qmemory.operate(ns.Z, 0)
                if meas_results[1]:
                    self.node.qmemory.operate(ns.X, 0)
                self.send_signal(Signals.SUCCESS, 0)
                fidelity = ns.qubits.fidelity(self.node.qmemory.peek(0)[0],
                                              ns.y0, squared=True)
                print(f"{ns.sim_time():.1f}: Bob received entangled qubit and "
                      f"corrections! Fidelity = {fidelity:.3f}")
                break

to finisih, take exmaple network from before, assign protocols and run the simulation
<br> if everything good, example_network_setup should return the network instead of the nodes and the connections (class Network)

In [75]:
from netsquid.examples.teleportation import example_network_setup

ns.sim_reset()
ns.set_qstate_formalism(ns.QFormalism.DM)
ns.set_random_state(seed=42)
network = example_network_setup()

alice = network.get_node("Alice")
bob = network.get_node("Bob")

random_state_protocol = InitStateProtocol(alice)
bell_measure_protocol = BellMeasurementProtocol(alice, random_state_protocol)
correction_protocol= CorrectionProtocol(bob)

bell_measure_protocol.start()
correction_protocol.start()
stats = ns.sim_run(100)

10.0 Alice received the entangled qubit, measured qubits & sending corrections
30.0: Bob received entangled qubit and corrections! Fidelity = 0.870
